In [2]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = "{:,.2f}".format

from experiments import *

import sys

sys.path.insert(1, '../fuzzylearn/')

from fuzzylearn import *
from fuzzylearn.fuzzifiers import LinearFuzzifier, CrispFuzzifier,ExponentialFuzzifier,QuantileLinearPiecewiseFuzzifier, QuantileConstantPiecewiseFuzzifier
from fuzzylearn.kernel import GaussianKernel, LinearKernel, HyperbolicKernel, PolynomialKernel, HomogeneousPolynomialKernel
from fuzzylearn import solve_optimization_gurobi

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, RobustScaler, PowerTransformer, Normalizer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score

dataset = pd.read_excel("dataset/IncidentiModificato.xlsx")
dataset = dataset.set_index("VERBALE")

In [3]:
import datetime as dt

dataset.DATA = dataset.DATA.apply(lambda d: (d - dt.datetime(1970,1,1)).days)

dataset.head()

,DATA,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,...,Scheletro:Rachide-cervicale,Scheletro:Rachide-toracico,Scheletro:Rachide-lombare,Scheletro:Bacino-e-sacro,Scheletro:Complesso-sterno/claveo/costale,Tot Testa,Tot Torace,Tot Addome,Tot Scheletro,Totale
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,10893,0,81,84.00,1.75,27.43,0,1,0,0,...,0,3,0,3,3,2,0,3,9,14
85829,10970,1,69,69.00,1.62,26.29,1,4,4,4,...,0,0,0,0,4,20,7,1,4,32
85977,11026,1,71,67.00,1.55,27.89,1,2,0,1,...,0,0,0,0,4,6,0,0,4,10
86220,11122,1,54,60.00,1.59,23.73,1,4,0,0,...,0,0,0,0,4,5,3,2,4,14
86247,11130,1,78,69.00,1.67,24.74,1,2,0,0,...,0,0,0,0,4,2,0,2,4,8


In [13]:
import logging

# create logger
f_logger = logging.getLogger(__name__)
f_logger.setLevel(logging.INFO)

# create console handler and set level to debug
file = logging.FileHandler('exp20.log')
file.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s -%(message)s')

# add formatter to ch
file.setFormatter(formatter)

# add ch to logger
f_logger.addHandler(file)

In [21]:
filejson = 'json_result/exp20.json'

In [5]:
import json
from experiments import *
from defuzzification import *

In [19]:
ordini = [.01,.1,1,10,100,1000]
pg_ordini = {
    'c': [.1],
    'k': [GaussianKernel(sigma) for sigma in ordini] + [HyperbolicKernel(1,o) for o in ordini] +
        [LinearKernel()]
}

In [7]:
# Totale, Totali, Lesioni5,10,15,20, Anagrafica+Lesioni

In [11]:
y = dataset['Mezzo'].values

Totale

In [17]:
tot = ['Totale']
tots = list(dataset.columns[-5:-1])
col_exp = [tot,tots]
labels = ['Totale','Totali']

In [10]:
ft = incidenti_fuzzifier_table(dataset,col_exp,fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=labels,\
                               file_json=filejson)

In [22]:
mu0 = [1 if e==0 else 0 for e in y]
labels0 = list(map(lambda x: x+"_0",labels))
ft = incidenti_fuzzifier_table(dataset,col_exp,fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),labels=labels0,\
                               file_json=filejson)

In [12]:
columns_lesioni = list(dataset.columns)[7:27]
labels = ['lesioniPCA5','lesioniPCA10','lesioniPCA15','lesioniPCA20']

In [13]:
for c,l in zip([5,10,15,20],labels):
    ft = incidenti_fuzzifier_table(dataset,[columns_lesioni],fuzzifiers_class,FuzzyInductor,y,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),dim_reduction=PCA(n_components=c),\
                                labels=[l],file_json=filejson)

In [14]:
labels0 = list(map(lambda x: x+"_0",labels))
for c,l in zip([5,10],labels0[0:2]):
    ft = incidenti_fuzzifier_table(dataset,[columns_lesioni],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),dim_reduction=PCA(n_components=c),\
                                labels=[l],file_json=filejson)

In [15]:
for c,l in zip([15,20],labels0[2:]):
    ft = incidenti_fuzzifier_table(dataset,[columns_lesioni],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),dim_reduction=PCA(n_components=c),\
                                labels=[l],file_json=filejson)

Nuovi esperimenti peso, altezza, BMI

In [6]:
offsets = list(range(0,6))
sigmas = [.1,.255,.3,.4,.5,.6,.7,.8,.9]
pg = {'c':[0.021544346900318846], 
      'k': [LinearKernel()] + [HyperbolicKernel(1,o) for o in offsets] + [GaussianKernel(sigma) for sigma in sigmas]
     }

In [7]:
columns_anagrafica = list(dataset.columns[1:5])
columns_lesioni = list(dataset.columns[7:27])

In [8]:
columns_peso = columns_lesioni + [col for col in columns_anagrafica if col != 'ALTEZZA']

In [14]:
l='PesoLesioni20'
dim_red = PCA(n_components=20)

ftp = incidenti_fuzzifier_table(dataset,[columns_peso],fuzzifiers_class,FuzzyInductor,y,pg,3,3,\
                                logger=f_logger,dim_reduction=dim_red,file_json='fuzzifiers.json',labels=[l])

In [23]:
l='PesoLesioni20_0'
ftp0 = incidenti_fuzzifier_table(dataset,[columns_peso],fuzzifiers_class,FuzzyInductor,mu0,pg,3,3,logger=f_logger,\
                                 dim_reduction=dim_red,file_json='fuzzifiers0.json',labels=[l])

In [24]:
columns_altezza = columns_lesioni + [col for col in columns_anagrafica if col != 'PESO']

In [25]:
l='AltezzaLesioni20'
dim_red = PCA(n_components=20)
fta = incidenti_fuzzifier_table(dataset,[columns_altezza],fuzzifiers_class,FuzzyInductor,y,pg,3,3,\
                                logger=f_logger,dim_reduction=dim_red,file_json='fuzzifiers.json',labels=[l])

In [26]:
l = 'AltezzaLesioni20_0'
fta0 = incidenti_fuzzifier_table(dataset,[columns_altezza],fuzzifiers_class,FuzzyInductor,mu0,pg,3,3,\
                                 logger=f_logger,dim_reduction=dim_red,file_json='fuzzifiers0.json',labels=[l])

In [28]:
BMI = ['BMI']
columns_BMI = columns_lesioni + BMI + [col for col in columns_anagrafica if col!='PESO' and col!='ALTEZZA']

In [29]:
l='BMILesioniNokgh20'
dim_red = PCA(n_components=20)

ftb = incidenti_fuzzifier_table(dataset,[columns_BMI],fuzzifiers_class,FuzzyInductor,mu0,pg,3,3,\
                                logger=f_logger,dim_reduction=dim_red,file_json='fuzzifiers.json',labels=[l])

In [31]:
l = 'BMILesioniNokgh20_0'
ftb0 = incidenti_fuzzifier_table(dataset,[columns_BMI],fuzzifiers_class,FuzzyInductor,mu0,pg,3,3,logger=f_logger,dim_reduction=dim_red,file_json='fuzzifiers0.json',labels=[l])

In [20]:
columns_anagrafica = list(dataset.columns[0:6])
ph = ['PESO','ALTEZZA']
columns = columns_lesioni + ph
label = 'PesoAltezzaLesioni20_0'
ft = incidenti_fuzzifier_table(dataset,[columns],fuzzifiers_class,FuzzyInductor,mu0,\
                               pg_ordini, 3,3,logger=f_logger,scaling=StandardScaler(),dim_reduction=PCA(n_components=20),\
                                labels=[label],file_json=filejson)

100%|██████████| 100/100 [00:57<00:00,  1.73it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:33<00:00,  2.95it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:30<00:00,  3.33it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:26<00:00,  3.75it/s]
/home/manueldileo/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 100/100 [00:44<00:00,  2.27it/s]
